# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [21]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [22]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [23]:
#your code here
spaceship = spaceship.dropna()
# Define a function that extracts the first letter
def first_letter(text):
    return text[0] if len(text) > 0 else ''

# Apply the function to the 'names' column
spaceship['Cabin'] = spaceship['Cabin'].apply(first_letter)
spaceship['Cabin'].value_counts()
spaceship = spaceship.drop(columns=['PassengerId', 'Name'])


non_numerical_cols = spaceship.select_dtypes(include=['object']).columns
spaceship_dummies = pd.get_dummies(spaceship, columns=non_numerical_cols)
target = spaceship_dummies['Transported']
features = spaceship_dummies.drop(columns=['Transported'])

**Perform Train Test Split**

In [24]:
#your code here
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)
normalizer = MinMaxScaler()
normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)
X_train.head()


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
3432,32.0,0.0,0.0,0.0,0.0,0.0,False,False,True,False,...,False,True,False,False,False,False,False,True,True,False
7312,4.0,0.0,0.0,0.0,0.0,0.0,True,False,False,False,...,False,False,False,True,False,False,False,True,True,False
2042,30.0,0.0,236.0,0.0,1149.0,0.0,True,False,False,True,...,False,False,True,False,False,False,False,True,True,False
4999,17.0,13.0,0.0,565.0,367.0,1.0,False,False,True,True,...,False,True,False,False,False,False,False,True,True,False
5755,26.0,0.0,0.0,0.0,0.0,0.0,True,False,False,False,...,False,False,False,True,False,False,False,True,True,False


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [31]:
#your code here
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

bagging_reg.fit(X_train_norm, y_train)
pred = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

MAE 0.2782674812898076
RMSE 0.37905247104073286
R2 score 0.4252768967916575


c:\Users\flori\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Random Forests

In [32]:
#your code here
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)
forest.fit(X_train_norm, y_train)
pred = forest.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", forest.score(X_test_norm, y_test))

MAE 0.2684322272876603
RMSE 0.3837822564533956
R2 score 0.41084471852616045


c:\Users\flori\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Gradient Boosting

In [34]:
#your code 

gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)
gb_reg.fit(X_train_norm, y_train)
pred = gb_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", gb_reg.score(X_test_norm, y_test))


MAE 0.26075407714237364
RMSE 0.41643750861049394
R2 score 0.3063192056891392


c:\Users\flori\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Adaptive Boosting

In [33]:
#your code here

ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)
ada_reg.fit(X_train_norm, y_train)

pred = ada_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test_norm, y_test))


MAE 0.24831318526142301
RMSE 0.42905532522960793
R2 score 0.2636461115684615


c:\Users\flori\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Which model is the best and why?

In [29]:

# Bagging and Pasting has the lowest RMSE, meaning the errors in predictions are relatively smaller overall, and the highest R², indicating it explains the variance in the data better than the other models.